# Task 1: Data Preparation


In [293]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

pd.set_option('future.no_silent_downcasting', True)

In [294]:
# csv to dataframe 
df = pd.read_csv("A1data.csv")

In [295]:
# get data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ISO3                       90 non-null     object
 1   Countries and areas        90 non-null     object
 2   Region                     90 non-null     object
 3   Sub-region                 90 non-null     object
 4   Income Group               90 non-null     object
 5   Total                      90 non-null     object
 6   Residence (Rural)          85 non-null     object
 7   Residence (Urban)          86 non-null     object
 8   Wealth quintile (Poorest)  83 non-null     object
 9   Wealth quintile (Richest)  83 non-null     object
 10  Time period                90 non-null     object
dtypes: object(11)
memory usage: 7.9+ KB


In [296]:
#find duplicate rows
print(df[df.duplicated()])

   ISO3 Countries and areas Region Sub-region              Income Group Total  \
30  GTM           Guatemala    LAC        LAC  Upper middle income (UM)   10%   
75  TGO                Togo    SSA        WCA            Low income (L)   24%   
76  TGO                Togo    SSA        WCA            Low income (L)   24%   

   Residence (Rural) Residence (Urban) Wealth quintile (Poorest)  \
30                3%               20%                        0%   
75                8%               49%                        0%   
76                8%               49%                        0%   

   Wealth quintile (Richest) Time period  
30                       49%   2014-2015  
75                       78%        2017  
76                       78%        2017  


In [297]:
#drop duplicates
df.drop_duplicates(inplace=True)
print(df[df.duplicated()])

Empty DataFrame
Columns: [ISO3, Countries and areas, Region, Sub-region, Income Group, Total, Residence (Rural), Residence (Urban), Wealth quintile (Poorest), Wealth quintile (Richest), Time period]
Index: []


In [298]:
# convert percentage strings to float
df.iloc[:, 5:10] = df.iloc[:, 5:10].apply(lambda x: x.str.replace('%', '') if x.dtype == "object" else x)
df.iloc[:, 5:10] = df.iloc[:, 5:10].astype(float)    

In [299]:
# finding structural error
df["Income Group"].value_counts()

Income Group
Upper middle income (UM)     32
Lower middle income (LM)     28
Low income (L)               18
High income (H)               7
Lower middle income (LLM)     1
Lower middle income (LMM)     1
Name: count, dtype: int64

In [300]:
# replace with correct category
df["Income Group"] = df["Income Group"].replace({
    "Lower middle income (LLM)": "Lower middle income (LM)",
    "Lower middle income (LMM)": "Lower middle income (LM)"
})
df["Income Group"].value_counts()

Income Group
Upper middle income (UM)    32
Lower middle income (LM)    30
Low income (L)              18
High income (H)              7
Name: count, dtype: int64

In [301]:
# finding structural error
df["Time period"].value_counts()

Time period
2018         16
2018-2019     9
2019          9
2017          8
2013          6
2012          5
2017-2018     5
2015          4
2015-2016     4
2014-2015     3
2010          3
2014          3
2016          3
2011-2012     3
2016-2017     2
2076          1
3562          1
2011          1
2012-2099     1
Name: count, dtype: int64

In [302]:
# remove impossible time periods
df = df[(df["Time period"] != '3562') & (df["Time period"] != '2012-2099')]
df["Time period"].value_counts()

Time period
2018         16
2018-2019     9
2019          9
2017          8
2013          6
2017-2018     5
2012          5
2015-2016     4
2015          4
2016          3
2010          3
2014          3
2011-2012     3
2014-2015     3
2016-2017     2
2076          1
2011          1
Name: count, dtype: int64

In [303]:
# remove impossible percentages
df = df.drop(df[(df["Total"] < 0) | (df["Total"] > 100)].index)
df = df.drop(df[(df["Residence (Rural)"] < 0) | (df["Residence (Rural)"] > 100)].index)
df = df.drop(df[(df["Wealth quintile (Poorest)"] < 0) | (df["Wealth quintile (Poorest)"] > 100)].index)
df = df.drop(df[(df["Wealth quintile (Richest)"] < 0) | (df["Wealth quintile (Richest)"] > 100)].index)

In [304]:
# columns with NaN values
df.isnull().sum()

ISO3                         0
Countries and areas          0
Region                       0
Sub-region                   0
Income Group                 0
Total                        0
Residence (Rural)            5
Residence (Urban)            4
Wealth quintile (Poorest)    7
Wealth quintile (Richest)    7
Time period                  0
dtype: int64

In [305]:
# get skewness values
print(df[["Residence (Rural)", "Residence (Urban)", "Wealth quintile (Poorest)", "Wealth quintile (Richest)"]].skew())

Residence (Rural)            1.021188
Residence (Urban)            0.302502
Wealth quintile (Poorest)    1.620091
Wealth quintile (Richest)   -0.455263
dtype: object


In [306]:
# get the mean values of each income group for columns with NaN values
income_group_means = df.groupby("Income Group")[["Residence (Rural)", "Residence (Urban)", "Wealth quintile (Richest)"]].transform('mean')
# median for Wealth quintile (Poorest) since skew is >1.5
income_group_medians = df.groupby("Income Group")[["Wealth quintile (Poorest)"]].transform('median')

In [307]:
income_group_means.head()

,Residence (Rural),Residence (Urban),Wealth quintile (Richest)
0,42.392857,61.928571,82.888889
1,11.862069,30.0,47.464286
2,42.392857,61.928571,82.888889
3,42.392857,61.928571,82.888889
4,11.862069,30.0,47.464286


In [308]:
income_group_medians.head()

,Wealth quintile (Poorest)
0,21.0
1,1.5
2,21.0
3,21.0
4,1.5


In [309]:
# replace Nan values with means/medians of that income group for the column
df["Residence (Rural)"] = df["Residence (Rural)"].fillna(income_group_means["Residence (Rural)"])
df["Residence (Urban)"] = df["Residence (Urban)"].fillna(income_group_means["Residence (Urban)"])
df["Wealth quintile (Richest)"] = df["Wealth quintile (Richest)"].fillna(income_group_means["Wealth quintile (Richest)"])
df["Wealth quintile (Poorest)"] = df["Wealth quintile (Poorest)"].fillna(income_group_medians["Wealth quintile (Poorest)"])

In [310]:
df.isnull().sum()

ISO3                         0
Countries and areas          0
Region                       0
Sub-region                   0
Income Group                 0
Total                        0
Residence (Rural)            0
Residence (Urban)            0
Wealth quintile (Poorest)    0
Wealth quintile (Richest)    0
Time period                  0
dtype: int64

In [311]:
#remove whitespace in countries and convert to title string
df["Countries and areas"] = df["Countries and areas"].apply(lambda x: x.replace(" ", ""))
df["Countries and areas"] = df["Countries and areas"].str.title()

In [315]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, 0 to 89
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ISO3                       84 non-null     object
 1   Countries and areas        84 non-null     object
 2   Region                     84 non-null     object
 3   Sub-region                 84 non-null     object
 4   Income Group               84 non-null     object
 5   Total                      84 non-null     object
 6   Residence (Rural)          84 non-null     object
 7   Residence (Urban)          84 non-null     object
 8   Wealth quintile (Poorest)  84 non-null     object
 9   Wealth quintile (Richest)  84 non-null     object
 10  Time period                84 non-null     object
dtypes: object(11)
memory usage: 7.9+ KB


,ISO3,Countries and areas,Region,Sub-region,Income Group,Total,Residence (Rural),Residence (Urban),Wealth quintile (Poorest),Wealth quintile (Richest),Time period
count,84,84,84,84,84,84.0,84.0,84.0,84.0,84.0,84
unique,84,84,6,8,4,51.0,45.0,56.0,36.0,55.0,17
top,DZA,Algeria,SSA,LAC,Upper middle income (UM),4.0,1.0,50.0,0.0,99.0,2018
freq,1,1,31,19,30,5.0,11.0,3.0,27.0,6.0,15


In [316]:
#write to csv
df.to_csv("S3902159-cleaned-A1data.csv")

# Task 2: Data Exploration

## Task 2.1 

In [ ]:
# Code goes after this line by adding cells


## Task 2.2 

In [ ]:
# Code goes after this line by adding cells


## Task 2.3 

In [ ]:
# Code goes after this line by adding cells
